In [1]:
import casadi as cs
from urdf2casadi import urdfparser as u2c
import urdf2casadi.geometry.dual_quaternion as dual_quaternion_geometry
import os # For current directory

In [2]:
dual_quaternion_to_transformation_matrix = dual_quaternion_geometry.to_numpy_transformation_matrix

In [3]:
# urdf2casadi uses cs.SX, which can be hard to read as these are sparse matrices.
# This short function just makes it so that the result will be a numpy matrix
# Use for 
def cs2np(asd):
    return cs.Function("temp",[],[asd])()["o0"].toarray()
# NOTE: casadi imports numpy as np, so cs.np is numpy

# Importing UR5 urdf

In [4]:
urdf_path = "../urdf/ur5_mod.urdf"
root_link = "base_link"
end_link = "tool0"
robot_parser = u2c.URDFparser()
robot_parser.from_file(urdf_path)
fk_dict = robot_parser.get_forward_kinematics(root_link, end_link)
print fk_dict.keys()


['q', 'upper', 'lower', 'dual_quaternion_fk', 'joint_names', 'T_fk', 'joint_list', 'quaternion_fk']


`fk_dict` is a python dictionary with some of the things we can extract from the URDF. In this example we will show the forward kinematics for the UR5 as a transformation matrix, as a dual quaternion, and a way of calculating the jacobians for the forward kinematics.

## Joint information

In [5]:
# CasADi SX symbol giving the joint symbols:
q = fk_dict["q"]
# Upper limits of the joint values
q_upper = fk_dict["upper"]
# Lower limits of the joint values
q_lower = fk_dict["lower"]
# Joint names
joint_names = fk_dict["joint_names"]
print("Number of joints:", q.size()[0])
print("Upper limits:", q_upper)
print("Lower limits:", q_lower)
print("Joint names:", joint_names)

('Number of joints:', 6)
('Upper limits:', [6.28318530718, 6.28318530718, 3.14159265359, 6.28318530718, 6.28318530718, 6.28318530718])
('Lower limits:', [-6.28318530718, -6.28318530718, -3.14159265359, -6.28318530718, -6.28318530718, -6.28318530718])
('Joint names:', ['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint'])


## Forward kinematics

In [6]:
# CasADi SX function for transformation matrix of the forward kinematics:
T_fk = fk_dict["T_fk"]
# CasADi SX function for dual_quaternion of the forward kinematics:
Q_fk = fk_dict["dual_quaternion_fk"]

So what's the position when all joint values are zero?

In [7]:
T0 = T_fk([0., 0., 0., 0., 0., 0.])
p0 = T0[:3, 3]
R0 = T0[:3, :3]
print("Transformation matrix:\n",cs2np(T0))
print("Position:\n", "x:",p0[0]," y:", p0[1], " z:", p0[2])
print("Distance from origin:\n", cs.np.linalg.norm(p0), "m")

('Transformation matrix:\n', array([[-1.00000000e+00, -9.79317772e-12,  4.79531649e-23,
         8.17250000e-01],
       [ 0.00000000e+00,  4.89658886e-12,  1.00000000e+00,
         1.91450000e-01],
       [-9.79317772e-12,  1.00000000e+00, -4.89658886e-12,
        -5.49100000e-03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]]))
('Position:\n', 'x:', DM(0.81725), ' y:', DM(0.19145), ' z:', DM(-0.005491))
('Distance from origin:\n', 0.8393931236807168, 'm')


And how about as a dual quaternion?

In [8]:
Q0 = Q_fk([0., 0., 0., 0., 0., 0.])
TofQ0 = dual_quaternion_to_transformation_matrix(Q0)
print("Dual quaternion:\n", cs2np(Q0))
print("Dual quaternion as transformation matrix:\n", cs2np(TofQ0))
if cs.np.linalg.norm(cs2np(TofQ0) - cs2np(T0)) < 1e-12:
    print("||TofQ0 - T0||< 1e-12, so they are equal")

('Dual quaternion:\n', array([[-3.46240714e-12],
       [ 7.07106781e-01],
       [ 7.07106781e-01],
       [ 3.46240714e-12],
       [ 6.96291583e-02],
       [-2.88941508e-01],
       [ 2.88941508e-01],
       [-6.57464350e-02]]))
('Dual quaternion as transformation matrix:\n', array([[-1.00000000e+00, -9.79316628e-12,  4.79538573e-23,
         8.17250000e-01],
       [ 0.00000000e+00,  4.89658314e-12,  1.00000000e+00,
         1.91450000e-01],
       [-9.79316628e-12,  1.00000000e+00, -4.89658314e-12,
        -5.49100000e-03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]]))
||TofQ0 - T0||< 1e-12, so they are equal


## Jacobians
As we're dealing with symbols, we formulate the symbolic expression for the jacobian, and then we create the functions for them.

In [9]:
# Create symbols
fk_position_jacobian_sym = cs.jacobian(T_fk(q)[:3,3], q)
fk_rotation_jacobian_sym = cs.jacobian(T_fk(q)[:3,:3], q)
fk_dual_quaternion_jacobian_sym = cs.jacobian(Q_fk(q), q)

In [10]:
# Create functions
fk_position_jacobian = cs.Function("jac_fk_pos", [q], [fk_position_jacobian_sym], ["q"], ["jac_fk_pos"])
fk_rotation_jacobian = cs.Function("jac_fk_rot", [q], [fk_rotation_jacobian_sym], ["q"], ["jac_fk_rot"])
fk_dual_quaternion_jacobian = cs.Function("jac_fk_Q", [q], [fk_dual_quaternion_jacobian_sym], ["q"], ["jac_fk_Q"])

Now let's test them out!

In [11]:
joint_vals = [0., 0., 0., 0., 0., 0.,]
pos_jac = fk_position_jacobian(joint_vals)
print("Positional jacobian at ", joint_vals, "is:\n", cs2np(pos_jac))

('Positional jacobian at ', [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'is:\n', array([[-1.91450000e-01, -9.46500000e-02, -9.46500000e-02,
        -9.46500000e-02,  8.23000000e-02,  0.00000000e+00],
       [ 8.17250000e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -8.17250000e-01, -3.92250000e-01,
        -9.26924271e-13,  8.05978526e-13,  0.00000000e+00]]))


What does this tell us? Let's look at each of the x, y, and z directions separately.

In [12]:
print("Comparative measure of how readily each direction is controlled:")
print("Norm of jac in x direction:", cs.np.linalg.norm(cs2np(pos_jac[0,:])))
print("Norm of jac in y direction:", cs.np.linalg.norm(cs2np(pos_jac[1,:])))
print("Norm of jac in z direction:", cs.np.linalg.norm(cs2np(pos_jac[2,:])))
print("\nComparative measure of how readily each joint affects positions:")
for i in range(len(joint_names)):
    print(joint_names[i]+":",cs.np.linalg.norm(cs2np(pos_jac[:,i])))

Comparative measure of how readily each direction is controlled:
('Norm of jac in x direction:', 0.265145733510609)
('Norm of jac in y direction:', 0.817250000000927)
('Norm of jac in z direction:', 0.9065084803807641)

Comparative measure of how readily each joint affects positions:
('shoulder_pan_joint:', 0.8393751634409462)
('shoulder_lift_joint:', 0.8227126989421991)
('elbow_joint:', 0.4035079738993563)
('wrist_1_joint:', 0.09465)
('wrist_2_joint:', 0.0823)
('wrist_3_joint:', 0.0)


So the Z direction is most affected by any of the joint values (but the last), motion in the T direction can only be done by the shoulder pan joint (`q[0]`) because all the motors are standing perpendicular to the y direction. Oh, and the last joint seems to do nothing, well, that's because it's the infinite rotational joint at the end of the UR robot that is just used for rotating the end-effector frame. So basically it doesn't change the position.

What are these comparative measures we're talking about? Essentially it amounts to asking, if we take the same tiny step on each of the joints, which one is the most affected, or if wish to move, which direction is most affect by a tiny step or set of tiny steps?

The following are not as tidy and easily read, but given for completion

In [13]:
joint_vals = [0., 0., 0., 0., 0., 0.,]
rot_jac = fk_rotation_jacobian(joint_vals)
print(cs2np(rot_jac))

[[ 0.00000000e+00 -9.79317772e-12 -9.79317772e-12 -9.79317772e-12
   0.00000000e+00 -9.79317772e-12]
 [-1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   0.00000000e+00  1.00000000e+00]
 [-4.89658886e-12  1.00000000e+00  1.00000000e+00  1.00000000e+00
   4.89658886e-12  1.00000000e+00]
 [-9.79317772e-12  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  9.79317772e-12  9.79317772e-12  9.79317772e-12
   4.79531649e-23  9.79317772e-12]
 [-1.00000000e+00 -4.89658886e-12 -4.89658886e-12 -4.89658886e-12
   1.00000000e+00 -4.89658886e-12]
 [ 4.79531649e-23  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -4.79531649e-23 -4.79531649e-23 -4.79531649e-23
   9.79317772e-12 -4.79531649e-23]]


In [14]:
joint_vals = [0., 0., 0., 0., 0., 0.,]
Q_jac = fk_dual_quaternion_jacobian(joint_vals)
print(cs2np(Q_jac))
print(cs2np(Q_jac)[:4,0])

[[-3.53553391e-01  3.53553391e-01  3.53553391e-01  3.53553391e-01
   3.53553391e-01  3.53553391e-01]
 [-1.73120357e-12  1.73120357e-12  1.73120357e-12  1.73120357e-12
  -1.73120357e-12  1.73120357e-12]
 [ 1.73120357e-12  1.73120357e-12  1.73120357e-12  1.73120357e-12
   1.73120357e-12  1.73120357e-12]
 [-3.53553391e-01 -3.53553391e-01 -3.53553391e-01 -3.53553391e-01
   3.53553391e-01 -3.53553391e-01]
 [ 1.44470754e-01  1.44470754e-01 -5.78943677e-03 -1.44470754e-01
   1.44470754e-01 -1.44470754e-01]
 [ 3.48145791e-02 -1.35075073e-03 -1.35075073e-03 -1.35075073e-03
   3.77577343e-03 -3.48145791e-02]
 [-3.28732175e-02 -6.63370459e-02 -6.63370459e-02 -6.63370459e-02
  -5.71713510e-03 -3.28732175e-02]
 [-1.44470754e-01  1.44470754e-01 -5.78943677e-03 -1.44470754e-01
  -1.44470754e-01 -1.44470754e-01]]
[-3.53553391e-01 -1.73120357e-12  1.73120357e-12 -3.53553391e-01]


In [15]:
print("Comparative measure of joint effect on rotation")
for i in range(len(joint_names)):
    print(joint_names[i]+":",cs.np.linalg.norm(cs2np(Q_jac)[:4,i])) # First four elements = rotation quaternion

Comparative measure of joint effect on rotation
('shoulder_pan_joint:', 0.5)
('shoulder_lift_joint:', 0.5)
('elbow_joint:', 0.5)
('wrist_1_joint:', 0.5)
('wrist_2_joint:', 0.5)
('wrist_3_joint:', 0.5)


Basically taking a tiny step on any of the joints is equally capable at causing rotation regardless of which of the joints we choose to move. 